In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('/home/mis/Sukhesh/Git/Sukhesh-Projects/Hotel Bookings/hotel_bookings.csv')

filter1 = (df['children']==0) & (df['adults']==0) & (df['babies']==0)
## Adults,babies & children cant be zero at a same time bcz booking couldn't be possible if these 3 attributes are 0 ..
df2 = df[~filter1]
data= df2.drop_duplicates()
data[['lead_time' , 'total_of_special_requests' , 'adr']].describe().T
## getting (mean, median , std , percentile) of above features
data = data.reset_index(drop=True)

dict_month = {'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12,
       'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6}

data['arrival_date_month_index'] = data['arrival_date_month'].map(dict_month)
data['arrival_date'] = data['arrival_date_year'].astype(str) + '-' + data['arrival_date_month_index'].astype(str) + '-' + data['arrival_date_day_of_month'].astype(str)
data['arrival_date'] = pd.to_datetime(data['arrival_date']).dt.date

In [ ]:
data = data[data['arrival_date'].astype(str)>='2016-01-01']
data.to_csv("/home/mis/Downloads/Hotel Bookings/proccessed_input_hotel_bookings.csv",index=False)

In [3]:
'''
print quantile values ..

'''

for col in ['lead_time' , 'total_of_special_requests' , 'adr']:
    print('feature name : {}'.format(col))

    for i in range(90,101,1):
        quantile_value = np.quantile(data[col] , q=i/100)
        print('{}th quantile value is {}'.format(i , quantile_value))
    print('\n')
    
'''
"adr" feature seems to have Outlier as 99th percentile value
is 261 but 100th percentile(max value) is 5400 ..
'''

feature name : lead_time
90th quantile value is 204.0
91th quantile value is 212.0
92th quantile value is 220.0
93th quantile value is 230.0
94th quantile value is 241.0
95th quantile value is 256.0
96th quantile value is 272.0
97th quantile value is 291.0
98th quantile value is 315.0
99th quantile value is 347.0
100th quantile value is 737.0


feature name : total_of_special_requests
90th quantile value is 2.0
91th quantile value is 2.0
92th quantile value is 2.0
93th quantile value is 2.0
94th quantile value is 2.0
95th quantile value is 2.0
96th quantile value is 2.0
97th quantile value is 3.0
98th quantile value is 3.0
99th quantile value is 3.0
100th quantile value is 5.0


feature name : adr
90th quantile value is 174.0
91th quantile value is 179.0
92th quantile value is 185.0
93th quantile value is 190.0
94th quantile value is 197.1
95th quantile value is 204.13300000000018
96th quantile value is 213.0
97th quantile value is 225.0
98th quantile value is 239.0
99th quantile value

'\n"adr" feature seems to have Outlier as 99th percentile value\nis 261 but 100th percentile(max value) is 5400 ..\n'

In [4]:
not_cancelled = data[data['is_canceled']==0]
country_wise_data = not_cancelled['country'].value_counts().reset_index()
country_wise_data.columns = ['country' , 'No of guests']


!pip install chart-studio
!pip install plotly

### establishing the entire set-up of Plotly..

import chart_studio.plotly as py
## chart_studio provides a web-service for hosting graphs!

import plotly.graph_objs as go
import plotly.express as px

from plotly.offline import download_plotlyjs , init_notebook_mode , plot , iplot

## iplot() when working in a Jupyter Notebook to
## display the plot in the Ipython notebook.

init_notebook_mode(connected=True)


# show on map

map_guest = px.choropleth(data_frame = country_wise_data ,
              locations= country_wise_data['country'] ,
              color=country_wise_data['No of guests'] ,
              hover_name=country_wise_data['country'] ,
              title= "Home country of Guests"

             )

map_guest.show()

'''Most guests are from Portugal and other countries in Europe'''


'Most guests are from Portugal and other countries in Europe'

In [5]:
category_pivot = pd.crosstab(index = data['reserved_room_type'] , columns=data['assigned_room_type'] , margins=True  )

'''
lets find meaningful insight from this :
for A category room , 56436 folks have reserved "A" & 45850 folks get assigned_room as "A".. & rest are unable to get !
for B category room , 996 folks have reserved "B" &  872 folks get assigned_room as "B".. & rest are unable to get !
'''

category_pivot

assigned_room_type,A,B,C,D,E,F,G,H,I,K,L,All
reserved_room_type,,,,,,,,,,,,
A,45850,892,1253,6402,1034,390,176,94,205,140,0,56436
B,106,872,0,5,2,2,8,0,0,1,0,996
C,5,2,866,6,4,2,10,9,10,0,0,914
D,295,27,32,15979,657,199,82,9,67,29,0,17376
E,15,2,6,22,5458,383,97,4,40,9,0,6036
F,6,14,0,4,31,2636,113,3,10,3,0,2820
G,5,1,2,0,4,14,1999,7,15,3,0,2050
H,0,0,0,1,0,0,10,579,6,0,0,596
L,1,1,1,0,0,1,0,1,0,0,1,6


In [6]:
## we will say just normalize over row , hence we need to pass normalize = 'index'
pivot_normalize = pd.crosstab(index = data['reserved_room_type'] , columns=data['assigned_room_type'] , margins=True ,normalize='index' ).round(2)*100

'''
Q.. Is any difference between assigned and reserved room type ?
Ans : Yes
'''
pivot_normalize

assigned_room_type,A,B,C,D,E,F,G,H,I,K,L
reserved_room_type,,,,,,,,,,,
A,81.0,2.0,2.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0
B,11.0,88.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
C,1.0,0.0,95.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
D,2.0,0.0,0.0,92.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0
E,0.0,0.0,0.0,0.0,90.0,6.0,2.0,0.0,1.0,0.0,0.0
F,0.0,0.0,0.0,0.0,1.0,93.0,4.0,0.0,0.0,0.0,0.0
G,0.0,0.0,0.0,0.0,0.0,1.0,98.0,0.0,1.0,0.0,0.0
H,0.0,0.0,0.0,0.0,0.0,0.0,2.0,97.0,1.0,0.0,0.0
L,17.0,17.0,17.0,0.0,0.0,17.0,0.0,17.0,0.0,0.0,17.0


In [7]:
# pie plot
fig = px.pie(data ,
      values = data['market_segment'].value_counts().values ,
      names = data['market_segment'].value_counts().index)
fig.show()